<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Applied Data Science: Capstone (Week 5)</font></h1>
<h1 align=center><font size = 4>Shopping Mall and Department Store Location Selection in New York</font></h1>

## A summary of data analysis and evaluation steps:

<ol>
<li>Build a dataframe of neighborhoods in New York, through the given link.</li>
<li>Obtain geographical coordinates of these neighborhoods.</li>
<li>Obtain the venue data for the neighborhoods from Foursquare API.</li>
<li>Explore and cluster the neighborhoods.</li>
<li>Examine Clusters and Select the best cluster to open a new shopping mall or department store.</li>
<li>Discuss the result and provide an improvement for the project.</li>
</ol>

## Download all the dependencies that we will need.

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

# All requested packages already installed.



## 1. Build a dataframe of neighborhoods in New York, through the given link.

#### Run a `wget` command and access the data.

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

#### Load and explore the data.

In [5]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
#The relevant dataset has the key 'features'.
neighborhoods_data = newyork_data['features']

#### Transform the data into a python dataframe.

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

# Then fill the dataframe by a for loop.
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

#### Examine the dataframe.

In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)
neighborhoods.head()

The dataframe has 5 boroughs and 306 neighborhoods.


,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


## 2. Obtain geographical coordinates of these neighborhoods.

#### Use geopy library to get the latitude and longitude values of New York City.

In [11]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a map of New York with neighborhoods superimposed on top.

In [12]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

#### Save the map as HTML file.

In [21]:
map_newyork.save('map_newyork.html')

## 3. Obtain the venue data for the neighborhoods from Foursquare API.

#### Define Foursquare Credentials and Version.

In [13]:
CLIENT_ID = 'TG2YPGXUB4BNJUAYTXVKDTNJUGJOZWRZVWZL2UROH2TOFH34' # your Foursquare ID
CLIENT_SECRET = '23WFOIVCCHHVHCEKINCN4II4P3YRINZTTD2RXXYWXJYPVYL2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TG2YPGXUB4BNJUAYTXVKDTNJUGJOZWRZVWZL2UROH2TOFH34
CLIENT_SECRET:23WFOIVCCHHVHCEKINCN4II4P3YRINZTTD2RXXYWXJYPVYL2


#### Now, for obtaining venue data, we choose top 100 venues that are in each neighbourhood within a radius of 500 meters. From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [16]:
#Create a function to extract venue data for each neighbourhood.
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Create a new dataframe for venue data.

In [17]:
newyork_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

#### Examine the venue dataframe

In [18]:
print(newyork_venues.shape)
newyork_venues.head()

(10263, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


#### Check how many venues were returned for each neighborhood

In [19]:
newyork_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,30,30,30,30,30,30
Annadale,14,14,14,14,14,14
Arden Heights,5,5,5,5,5,5
Arlington,4,4,4,4,4,4
Arrochar,21,21,21,21,21,21
Arverne,18,18,18,18,18,18
Astoria,100,100,100,100,100,100
Astoria Heights,13,13,13,13,13,13
Auburndale,18,18,18,18,18,18


#### Find out how many unique categories can be curated from all the returned venues

In [20]:
print('There are {} uniques categories.'.format(len(newyork_venues['Venue Category'].unique())))

There are 431 uniques categories.


In [24]:
# print out the list of categories
newyork_venues['Venue Category'].unique()[:50]

array(['Dessert Shop', 'Pharmacy', 'Ice Cream Shop', 'Donut Shop',
       'Gas Station', 'Caribbean Restaurant', 'Sandwich Place', 'Food',
       'Laundromat', 'Discount Store', 'Pizza Place', 'Mattress Store',
       'Bagel Shop', 'Grocery Store', 'Fast Food Restaurant',
       'Restaurant', 'Chinese Restaurant', 'Gift Shop', 'Bus Station',
       'Basketball Court', 'Park', 'Accessories Store', 'Diner',
       'Seafood Restaurant', 'Deli / Bodega', 'Bowling Alley', 'Platform',
       'Juice Bar', 'Bus Stop', 'Bakery', 'Cosmetics Shop', 'Plaza',
       'River', 'Bank', 'Locksmith', 'Baseball Field', 'Food Truck',
       'Home Service', 'Gym', 'Playground', 'Gourmet Shop',
       'Latin American Restaurant', 'Pub', 'Beer Bar', 'Burger Joint',
       'Warehouse Store', 'Mexican Restaurant', 'Spanish Restaurant',
       'Coffee Shop', 'Bar'], dtype=object)

In [41]:
# check if the results contain "Shopping Mall" or "Department Store"
print("Shopping Mall" in newyork_venues['Venue Category'].unique())
print("Department Store" in newyork_venues['Venue Category'].unique())

True
True


## 4. Explore and cluster the neighborhoods.

In [22]:
#Analyze each neighbourhood for venues.
newyork_onehot = pd.get_dummies(newyork_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
newyork_onehot['Neighborhood'] = newyork_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [newyork_onehot.columns[-1]] + list(newyork_onehot.columns[:-1])
newyork_onehot = newyork_onehot[fixed_columns]

newyork_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle Hous

In [27]:
#Examine the new dataframe size.
newyork_onehot.shape

(10263, 431)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [33]:
newyork_grouped = newyork_onehot.groupby('Neighborhood').mean().reset_index()

print('The shape of the new dataframe:', newyork_grouped.shape)
newyork_grouped.head()

The shape of the new dataframe: (300, 431)


,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Gym,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle Hous

In [42]:
#Check neighbourhoods with shopping malls venue
print(len(newyork_grouped[newyork_grouped["Shopping Mall"] > 0]))
print(len(newyork_grouped[newyork_grouped["Department Store"] > 0]))

16
25


#### Create a new DataFrame for Shopping Mall data only

In [43]:
newyork_mall = newyork_grouped[["Neighborhood", "Shopping Mall", "Department Store"]]
newyork_mall.head()

,Neighborhood,Shopping Mall,Department Store
0,Allerton,0.0,0.033333
1,Annadale,0.0,0.000000
2,Arden Heights,0.0,0.000000
3,Arlington,0.0,0.000000
4,Arrochar,0.0,0.000000


#### Cluster Neighborhoods. Run k-means to cluster the neighborhood into 4 clusters.

In [51]:
# set number of clusters
kclusters = 4

newyork_clustering = newyork_mall.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(newyork_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int32)

In [52]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
newyork_merged = newyork_mall.copy()

# add clustering labels
newyork_merged["Cluster Labels"] = kmeans.labels_

In [53]:
newyork_merged.head()

,Neighborhood,Shopping Mall,Department Store,Cluster Labels
0,Allerton,0.0,0.033333,3
1,Annadale,0.0,0.000000,0
2,Arden Heights,0.0,0.000000,0
3,Arlington,0.0,0.000000,0
4,Arrochar,0.0,0.000000,0


In [54]:
# merge newyork_grouped with neighborhoods to add latitude/longitude for each neighborhood
newyork_merged = newyork_merged.join(neighborhoods.set_index("Neighborhood"), on="Neighborhood")

print(newyork_merged.shape)
newyork_merged.head() # check the last columns!

(304, 7)


,Neighborhood,Shopping Mall,Department Store,Cluster Labels,Borough,Latitude,Longitude
0,Allerton,0.0,0.033333,3,Bronx,40.865788,-73.859319
1,Annadale,0.0,0.000000,0,Staten Island,40.538114,-74.178549
2,Arden Heights,0.0,0.000000,0,Staten Island,40.549286,-74.185887
3,Arlington,0.0,0.000000,0,Staten Island,40.635325,-74.165104
4,Arrochar,0.0,0.000000,0,Staten Island,40.596313,-74.067124


In [55]:
# sort the results by Cluster Labels
print(newyork_merged.shape)
newyork_merged.sort_values(["Cluster Labels"], inplace=True)
newyork_merged.head()

(304, 7)


,Neighborhood,Shopping Mall,Department Store,Cluster Labels,Borough,Latitude,Longitude
299,Yorkville,0.0,0.0,0,Manhattan,40.775930,-73.947118
195,North Side,0.0,0.0,0,Brooklyn,40.714823,-73.958809
194,North Riverdale,0.0,0.0,0,Bronx,40.908543,-73.904531
193,North Corona,0.0,0.0,0,Queens,40.754071,-73.857518
192,Noho,0.0,0.0,0,Manhattan,40.723259,-73.988434


#### Finally, visualize the resulting clusters

In [56]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(newyork_merged['Latitude'], newyork_merged['Longitude'], newyork_merged['Neighborhood'], newyork_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## 5. Examine Clusters and Select the best cluster to open a new shopping mall or department store.

In [59]:
#Cluster 0
#It should be noted that Cluster does not either have shopping malls or department stores. Thus, we only list top 5
#rows since they all have values 0.
newyork_merged.loc[newyork_merged['Cluster Labels'] == 0].head()

,Neighborhood,Shopping Mall,Department Store,Cluster Labels,Borough,Latitude,Longitude
299,Yorkville,0.0,0.0,0,Manhattan,40.775930,-73.947118
195,North Side,0.0,0.0,0,Brooklyn,40.714823,-73.958809
194,North Riverdale,0.0,0.0,0,Bronx,40.908543,-73.904531
193,North Corona,0.0,0.0,0,Queens,40.754071,-73.857518
192,Noho,0.0,0.0,0,Manhattan,40.723259,-73.988434


### Observation: it seems cluster 0 neighbourhoods neither have shopping malls or department stores currently.

In [60]:
#Cluster 1
newyork_merged.loc[newyork_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Department Store,Cluster Labels,Borough,Latitude,Longitude
130,Hollis,0.153846,0.0,1,Queens,40.711243,-73.759250
259,Starrett City,0.090909,0.0,1,Brooklyn,40.647589,-73.879370
126,Heartland Village,0.111111,0.0,1,Staten Island,40.589139,-74.159022
7,Astoria Heights,0.076923,0.0,1,Queens,40.770317,-73.894680


### Observation: it seems cluster 1 neighbourhoods only have shopping malls but no department store currently.

In [61]:
#Cluster 2
newyork_merged.loc[newyork_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Department Store,Cluster Labels,Borough,Latitude,Longitude
256,St. Albans,0.062500,0.000000,2,Queens,40.694445,-73.758676
10,Battery Park City,0.032609,0.000000,2,Manhattan,40.711932,-74.016869
64,Concourse Village,0.025641,0.000000,2,Bronx,40.824780,-73.915847
109,Georgetown,0.033333,0.000000,2,Brooklyn,40.623845,-73.916075
142,Jamaica Hills,0.037037,0.000000,2,Queens,40.711460,-73.796465
191,New Springville,0.047619,0.000000,2,Staten Island,40.594252,-74.164960
48,Charleston,0.034483,0.034483,2,Staten Island,40.530531,-74.232158
166,Marble Hill,0.041667,0.041667,2,Manhattan,40.876551,-73.910660


### Observation: it seems (some of) cluster 2 neighbourhoods have both shopping malls and department stores, but neither of them is as dense as Cluster 1.

In [62]:
#Cluster 3
newyork_merged.loc[newyork_merged['Cluster Labels'] == 3]

,Neighborhood,Shopping Mall,Department Store,Cluster Labels,Borough,Latitude,Longitude
0,Allerton,0.00,0.033333,3,Bronx,40.865788,-73.859319
207,Parkchester,0.00,0.029412,3,Bronx,40.837938,-73.856003
203,Ozone Park,0.00,0.027778,3,Queens,40.680708,-73.843203
170,Melrose,0.00,0.037037,3,Bronx,40.819754,-73.909422
140,Jamaica Center,0.00,0.065217,3,Queens,40.704657,-73.796902
111,Glen Oaks,0.00,0.040000,3,Queens,40.749441,-73.715481
110,Gerritsen Beach,0.00,0.055556,3,Brooklyn,40.590848,-73.930102
77,East Flatbush,0.00,0.100000,3,Brooklyn,40.641718,-73.936103
287,Westchester Square,0.00,0.032258,3,Bronx,40.840619,-73.842194
149,Lefrak City,0.04,0.080000,3,Queens,40.736075,-73.862525


### Observation: it seems cluster 3 neighbourhoods have reletively less shopping malls and more department stores, compared with Cluster 1 and 2 neighbourhoods.

## Results based on observations:
<ol>
<li>For the neighbourhoods belonging to Cluster 0, from the perspective of competition due to oversupply and high concentration, both shopping malls and department stores can be considered.</li>
<li>For the neighbourhoods belonging to Cluster 1, investing on a department store seems to have less pressure from the peer competition.</li>
<li>For the neighbourhoods belonging to Cluster 3, investing on a shopping mall seems to have less pressure from the peer competition.</li>
<li>For the neighbourhoods belonging to Cluster 2, the investment condition seems like a intermediate case of Case 2 and Case 3 above.</li>
</ol>

## 6. Discuss the result and provide an improvement for the project.

## Based on the results above, there are still a few more factors that needs considering.
<ol>
<li>Considering the reality factors, the safety scaling data and the average income of neighbourhood residents need to be taken into consideration. Thus, from this perspective, whether cluster 0 is suitable for investment on shopping malls or department stores cannot be determined immediately.</li>
<li>For neighourhoods that already have shopping malls and department stores, the market has also come into a good form. That implies the safety and the consumption do not influence the decision very much. Thus, if investing shopping malls or department store there, we need more data about what features these shopping malls or department stores already have. Then after analysis, we can make the decision.</li>
<li>From the outcome of Cluster 3, there area still some neighbourhoods that do not have department stores. Hence, K=4 may not be the optimal value. We can run a few more cases and find the best K for analysis.</li>
</ol>